In [1]:
%cd ..

/home/qwj/code/HDInstruct


## Problems We have

前面出的问题大多是在correctness上，主要是“格式”的问题。生成response和ground truth不匹配；
uncertainty的输入都是模型的response，格式基本一致，不会有不一致的问题。

In [2]:
import os
import pandas as pd

In [3]:
# directories

chat_outputs_dir = 'outputs/chat_zero_shot_debug_2024-03-08_10-17'
completion_outputs_dir = 'outputs/completion_few_shot_debug2024-03-08_09-32'

file_names = {
    'with_context': 'coqa_responses.json',
    'without_context': 'nq_responses.json',
}

In [4]:
from rouge_score.rouge_scorer import RougeScorer
rouge_scorer = RougeScorer(['rougeL'], use_stemmer=True)

## Chat
mistral7b

### Context QA (CoQA)

In [5]:
chat_context_df = pd.read_json(os.path.join(chat_outputs_dir, file_names['with_context']))
chat_context_df.drop(columns=['id'], inplace=True)
chat_context_df.head()

,context,question,ground_truth,prompt,greedy_response,sampling_response
0,CHAPTER XXXII \n\nLITTLE NELL--CONCLUSION \n\n...,Was an arrangement made for those left in the ...,"[Yes, yes]","Based on the following passage, answer the que...","Yes, an arrangement was made for those remain...","{' Yes, an arrangement was made for those who ..."
1,This year marks the 100thanniversary of Leo To...,Who is helping to make a film about the author...,[Andrei Deryabin],"Based on the following passage, answer the que...",Andrei Deryabin and his team are producing th...,{' Andrei Deryabin and his team are producing ...
2,Harry is a boy with a learning disability. On ...,and what did he say?,"[""dog"" and ""mummy""., Dog and mummy, ""dog"", ""do...","Based on the following passage, answer the que...","Harry said ""dog"" and ""mummy"" for the first ti...","{' Harry said ""dog"" and ""mummy"" for the first ..."
3,The University of Kansas (KU) is a public rese...,did Kansas have its own civil war?,"[Yes, yes]","Based on the following passage, answer the que...","Yes, Kansas experienced a period of violent c...","{' Yes, it is referred to as ""Bleeding Kansas""..."
4,(CNN) -- Kenyan actress Lupita Nyong'o has bec...,What movie was her breakout role in?,"[12 Years A Slave, ""12 Years A Slave."", ""12 Ye...","Based on the following passage, answer the que...","""12 Years A Slave""","{' ""12 Years A Slave""': 20}"


In [6]:
new_chat_context_df = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in chat_context_df.iterrows():
    scores = rouge_scorer.score(target=row['ground_truth'][0], prediction=row['greedy_response'])
    new_row = {
        'question': row['question'], 
        'ground_truth': row['ground_truth'][0],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_chat_context_df.loc[index] = new_row

new_chat_context_df.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,Was an arrangement made for those left in the ...,Yes,"Yes, an arrangement was made for those remain...",0.064516,0.033333,1.0
1,Who is helping to make a film about the author...,Andrei Deryabin,Andrei Deryabin and his team are producing th...,0.200000,0.111111,1.0
2,and what did he say?,"""dog"" and ""mummy"".","Harry said ""dog"" and ""mummy"" for the first ti...",0.352941,0.214286,1.0
3,did Kansas have its own civil war?,Yes,"Yes, Kansas experienced a period of violent c...",0.035088,0.017857,1.0
4,What movie was her breakout role in?,12 Years A Slave,"""12 Years A Slave""",1.000000,1.000000,1.0


In [7]:
new_chat_context_df['rougeL_fmeasure'].mean()

0.2689968940796722

### QA (NQ)

In [8]:
qa_chat_df = pd.read_json(os.path.join(chat_outputs_dir, file_names['without_context']))
qa_chat_df.drop(columns=['id'], inplace=True)
qa_chat_df.head()

,question,ground_truth,prompt,greedy_response,sampling_response
0,where are the giant redwoods located in califo...,"[Humboldt County, Del Norte County, the coast ...",Answer the question with a short word or phras...,The giant redwoods are primarily located in n...,{' The giant redwoods are primarily located in...
1,who wrote old flames cant hold a candle to you,"[Pebe Sebert, Pebe Sebert and Hugh Moffatt, Pa...",Answer the question with a short word or phras...,"The phrase ""Old flames can't hold a candle to...","{' The phrase ""Old flames can't hold a candle ..."
2,who played the creature on jeepers creepers 3,[Jonathan Breck],Answer the question with a short word or phras...,"The creature in ""Jeepers Creepers 3"" was port...","{' The creature in ""Jeepers Creepers 3"" was po..."
3,what is the emblematic rhythm of dominican rep...,"[merengue, bachata]",Answer the question with a short word or phras...,Merengue is the emblematic rhythm of the Domi...,{' Merengue is the emblematic rhythm of the Do...
4,what type of rock dominates the earth's surface,"[Sedimentary, sedimentary rock, sedimentary]",Answer the question with a short word or phras...,The most common type of rock found on the Ear...,{' The most common type of rock found on the E...


In [9]:
new_qa_chat_df = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in qa_chat_df.iterrows():
    scores = rouge_scorer.score(target=row['ground_truth'][0], prediction=row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'][0],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_qa_chat_df.loc[index] = new_row
new_qa_chat_df.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,where are the giant redwoods located in califo...,Humboldt County,The giant redwoods are primarily located in n...,0.121212,0.064516,1.0
1,who wrote old flames cant hold a candle to you,Pebe Sebert,"The phrase ""Old flames can't hold a candle to...",0.000000,0.000000,0.0
2,who played the creature on jeepers creepers 3,Jonathan Breck,"The creature in ""Jeepers Creepers 3"" was port...",0.041667,0.021739,0.5
3,what is the emblematic rhythm of dominican rep...,merengue,Merengue is the emblematic rhythm of the Domi...,0.200000,0.111111,1.0
4,what type of rock dominates the earth's surface,Sedimentary,The most common type of rock found on the Ear...,0.125000,0.066667,1.0


In [10]:
new_qa_chat_df['rougeL_fmeasure'].mean()

0.10640686099962612

## Completion

### ContextQA

In [11]:
context_completion_df = pd.read_json(os.path.join(completion_outputs_dir, file_names['with_context']))
context_completion_df.drop(columns=['id'], inplace=True)
context_completion_df.head()

,context,question,ground_truth,prompt,greedy_response,sampling_response
0,CHAPTER XXXII \n\nLITTLE NELL--CONCLUSION \n\n...,Was an arrangement made for those left in the ...,"[Yes, yes]","Based on the following passage, answer the que...",Yes\n\n,"{' Yes ': 8, ' Yes': 3, ' Yes. ': 3, ' Yes, ..."
1,This year marks the 100thanniversary of Leo To...,Who is helping to make a film about the author...,[Andrei Deryabin],"Based on the following passage, answer the que...",Andrei Deryabin\n\n,"{' Andrei Deryabin ': 19, ' Andrey Deryabin ..."
2,Harry is a boy with a learning disability. On ...,and what did he say?,"[""dog"" and ""mummy""., Dog and mummy, ""dog"", ""do...","Based on the following passage, answer the que...","""dog"" and ""mummy""\n\n","{' ""dog"" and ""mummy"" ': 2, ' ""Dog"" and ""Mummy..."
3,The University of Kansas (KU) is a public rese...,did Kansas have its own civil war?,"[Yes, yes]","Based on the following passage, answer the que...",Yes\n\n,"{' Yes ': 4, ' yes ': 3, ' No ': 7, ' no '..."
4,(CNN) -- Kenyan actress Lupita Nyong'o has bec...,What movie was her breakout role in?,"[12 Years A Slave, ""12 Years A Slave."", ""12 Ye...","Based on the following passage, answer the que...",12 Years A Slave\n\n,"{' 12 Years A Slave ': 15, ' 12 Years A Slave..."


In [12]:
new_context_completion_df = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in context_completion_df.iterrows():
    scores = rouge_scorer.score(target=row['ground_truth'][0], prediction=row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'][0],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_context_completion_df.loc[index] = new_row
new_context_completion_df.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,Was an arrangement made for those left in the ...,Yes,Yes\n\n,1.0,1.0,1.0
1,Who is helping to make a film about the author...,Andrei Deryabin,Andrei Deryabin\n\n,1.0,1.0,1.0
2,and what did he say?,"""dog"" and ""mummy"".","""dog"" and ""mummy""\n\n",1.0,1.0,1.0
3,did Kansas have its own civil war?,Yes,Yes\n\n,1.0,1.0,1.0
4,What movie was her breakout role in?,12 Years A Slave,12 Years A Slave\n\n,1.0,1.0,1.0


In [13]:
new_context_completion_df['rougeL_fmeasure'].mean()

0.5963137253233444

### QA

In [14]:
qa_comletion_df = pd.read_json(os.path.join(completion_outputs_dir, file_names['without_context']))
qa_comletion_df.drop(columns=['id'], inplace=True)
qa_comletion_df.head()

,question,ground_truth,prompt,greedy_response,sampling_response
0,where are the giant redwoods located in califo...,"[Humboldt County, Del Norte County, the coast ...",Answer these questions:\n\nQ: In Scotland a bo...,Sequoia National Park\n\n,"{' Sequoia National Park ': 2, ' california ..."
1,who wrote old flames cant hold a candle to you,"[Pebe Sebert, Pebe Sebert and Hugh Moffatt, Pa...",Answer these questions:\n\nQ: In Scotland a bo...,John Denver\n\n,"{' Johnny Mercer ': 2, ' Dolly Parton ': 2, ..."
2,who played the creature on jeepers creepers 3,[Jonathan Breck],Answer these questions:\n\nQ: In Scotland a bo...,Brandon Smith\n\n,"{' Brandon Smith ': 3, ' Brandon Molale ': 4..."
3,what is the emblematic rhythm of dominican rep...,"[merengue, bachata]",Answer these questions:\n\nQ: In Scotland a bo...,merengue\n\n,"{' merengue ': 11, ' Merengue ': 7, ' mering..."
4,what type of rock dominates the earth's surface,"[Sedimentary, sedimentary rock, sedimentary]",Answer these questions:\n\nQ: In Scotland a bo...,Igneous\n\n,"{' Igneous ': 7, ' igneous ': 1, ' Sedimenta..."


In [15]:
new_qa_comletion_df = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in qa_comletion_df.iterrows():
    scores = rouge_scorer.score(target=row['ground_truth'][0], prediction=row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'][0],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_qa_comletion_df.loc[index] = new_row

new_qa_comletion_df.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,where are the giant redwoods located in califo...,Humboldt County,Sequoia National Park\n\n,0.0,0.0,0.0
1,who wrote old flames cant hold a candle to you,Pebe Sebert,John Denver\n\n,0.0,0.0,0.0
2,who played the creature on jeepers creepers 3,Jonathan Breck,Brandon Smith\n\n,0.0,0.0,0.0
3,what is the emblematic rhythm of dominican rep...,merengue,merengue\n\n,1.0,1.0,1.0
4,what type of rock dominates the earth's surface,Sedimentary,Igneous\n\n,0.0,0.0,0.0


In [16]:
new_qa_comletion_df['rougeL_fmeasure'].mean()

0.31085714285714283

---

综上，completion model就是个好东西；没必要复杂化成instruct model

## 对整个gt aliases 做检查

In [17]:
from rouge_score.scoring import Score

def compute_rouge(target_candidates, prediction):
    targets = target_candidates.copy()
    best_score = {'rougeL': Score(precision=0, recall=0, fmeasure=0)}
    while targets:
        target = targets.pop()
        score = rouge_scorer.score(target=target, prediction=prediction)
        if score['rougeL'].fmeasure > best_score['rougeL'].fmeasure:
            best_score = score
    return best_score

### chat

In [18]:
new_chat_context_df_2 = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in chat_context_df.iterrows():
    scores = compute_rouge(row['ground_truth'], row['greedy_response'])
    new_row = {
        'question': row['question'], 
        'ground_truth': row['ground_truth'],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_chat_context_df_2.loc[index] = new_row

new_chat_context_df_2.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,Was an arrangement made for those left in the ...,"[Yes, yes]","Yes, an arrangement was made for those remain...",0.064516,0.033333,1.0
1,Who is helping to make a film about the author...,[Andrei Deryabin],Andrei Deryabin and his team are producing th...,0.200000,0.111111,1.0
2,and what did he say?,"[""dog"" and ""mummy""., Dog and mummy, ""dog"", ""do...","Harry said ""dog"" and ""mummy"" for the first ti...",0.352941,0.214286,1.0
3,did Kansas have its own civil war?,"[Yes, yes]","Yes, Kansas experienced a period of violent c...",0.035088,0.017857,1.0
4,What movie was her breakout role in?,"[12 Years A Slave, ""12 Years A Slave."", ""12 Ye...","""12 Years A Slave""",1.000000,1.000000,1.0


In [19]:
new_chat_context_df['rougeL_fmeasure'].mean(), new_chat_context_df_2['rougeL_fmeasure'].mean()

(0.2689968940796722, 0.2946655621902916)

In [20]:
new_qa_chat_df_2 = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in qa_chat_df.iterrows():
    scores = compute_rouge(row['ground_truth'], row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_qa_chat_df_2.loc[index] = new_row
new_qa_chat_df_2.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,where are the giant redwoods located in califo...,"[Humboldt County, Del Norte County, the coast ...",The giant redwoods are primarily located in n...,0.176471,0.096774,1.0
1,who wrote old flames cant hold a candle to you,"[Pebe Sebert, Pebe Sebert and Hugh Moffatt, Pa...","The phrase ""Old flames can't hold a candle to...",0.000000,0.000000,0.0
2,who played the creature on jeepers creepers 3,[Jonathan Breck],"The creature in ""Jeepers Creepers 3"" was port...",0.041667,0.021739,0.5
3,what is the emblematic rhythm of dominican rep...,"[merengue, bachata]",Merengue is the emblematic rhythm of the Domi...,0.200000,0.111111,1.0
4,what type of rock dominates the earth's surface,"[Sedimentary, sedimentary rock, sedimentary]",The most common type of rock found on the Ear...,0.235294,0.133333,1.0


In [21]:
new_qa_chat_df['rougeL_fmeasure'].mean(), new_qa_chat_df_2['rougeL_fmeasure'].mean()

(0.10640686099962612, 0.13223958264880742)

### completion

In [22]:
new_context_completion_df_2 = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in context_completion_df.iterrows():
    scores = compute_rouge(row['ground_truth'], row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_context_completion_df_2.loc[index] = new_row
new_context_completion_df_2.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,Was an arrangement made for those left in the ...,"[Yes, yes]",Yes\n\n,1.0,1.0,1.0
1,Who is helping to make a film about the author...,[Andrei Deryabin],Andrei Deryabin\n\n,1.0,1.0,1.0
2,and what did he say?,"[""dog"" and ""mummy""., Dog and mummy, ""dog"", ""do...","""dog"" and ""mummy""\n\n",1.0,1.0,1.0
3,did Kansas have its own civil war?,"[Yes, yes]",Yes\n\n,1.0,1.0,1.0
4,What movie was her breakout role in?,"[12 Years A Slave, ""12 Years A Slave."", ""12 Ye...",12 Years A Slave\n\n,1.0,1.0,1.0


In [23]:
new_context_completion_df['rougeL_fmeasure'].mean(), new_context_completion_df_2['rougeL_fmeasure'].mean()

(0.5963137253233444, 0.6983562837207415)

In [24]:
new_qa_comletion_df_2 = pd.DataFrame(columns=['question', 'ground_truth', 'greedy_response', 'rougeL_fmeasure', 'rougeL_precision', 'rougeL_recall'])
for index, row in qa_comletion_df.iterrows():
    scores = compute_rouge(row['ground_truth'], row['greedy_response'])
    new_row = {
        'question': row['question'],
        'ground_truth': row['ground_truth'],
        'greedy_response': row['greedy_response'],
        'rougeL_fmeasure': scores['rougeL'].fmeasure,
        'rougeL_precision': scores['rougeL'].precision,
        'rougeL_recall': scores['rougeL'].recall
    }
    new_qa_comletion_df_2.loc[index] = new_row
new_qa_comletion_df_2.head()

,question,ground_truth,greedy_response,rougeL_fmeasure,rougeL_precision,rougeL_recall
0,where are the giant redwoods located in califo...,"[Humboldt County, Del Norte County, the coast ...",Sequoia National Park\n\n,0.0,0.0,0.0
1,who wrote old flames cant hold a candle to you,"[Pebe Sebert, Pebe Sebert and Hugh Moffatt, Pa...",John Denver\n\n,0.0,0.0,0.0
2,who played the creature on jeepers creepers 3,[Jonathan Breck],Brandon Smith\n\n,0.0,0.0,0.0
3,what is the emblematic rhythm of dominican rep...,"[merengue, bachata]",merengue\n\n,1.0,1.0,1.0
4,what type of rock dominates the earth's surface,"[Sedimentary, sedimentary rock, sedimentary]",Igneous\n\n,0.0,0.0,0.0


In [25]:
new_qa_comletion_df['rougeL_fmeasure'].mean(), new_qa_comletion_df_2['rougeL_fmeasure'].mean()

(0.31085714285714283, 0.38150476190476185)